In [ ]:
# 구글 웹검색 결과를 제공하는 serper dev

In [10]:
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
from crewai import Agent, Task, Crew
from crewai.process import Process
# serp의 apikey를 가져와야 한다.
from crewai_tools import (
    SerperDevTool, # 특정 키워드를 줬을때, 해당 keyword를 구글에 검색한 결과를 가져오는 도구
    WebsiteSearchTool, # 특정 키워드, 웹사이트를 주면, 해당 웹사이트 내에서 해당 키워드와 관련이 있는 부분만 가져오는 RAG가 내부적으로 실행이 되는 도구
    ScrapeWebsiteTool # 해당 URL에 있는 모든 텍스트를 scrape해서 가져오는 도구. 일종의 crawling을 하는 도구다.
    
)

In [12]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')

search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()
scrap_tool = ScrapeWebsiteTool()

# agent
researcher = Agent(
    role="테크 트렌드 연구원",
    goal="인공지능 분야의 최신 기술 트랜드를 한국어로 제공합니다. 지금은 2024년 9월입니다.",
    backstory="기술 트렌드에 예리한 한목을 지닌 분석가이자 AI개발자입니다.",
    tools=[search_tool, web_rag_tool],
    verbose=True, # AI출력물들의 중간 결과값들
    max_iter=5, # 해당 agent가 주어진 task를 수행하는데 있어서 반복실행한 결과를 제한한다. 제한을 주지 않으면 계속해서 search하여 token수가 많이 나오게 된다.
    llm = llm
)

writer = Agent(
    role="뉴스레터 작성자",
    goal="최신 AI기술 트렌드에 대한 매력적인 테크 뉴스레터를 한국어로 작성하세요. 지금은 2024년 9월입니다.",
    backstory="기술에 대한 열정을 가진 숙련된 작가입니다.",
    verbose=True,
    allow_delegation=False, # True인경우, 해당 agent가 주어진 task를 수행하는데 있어서, 어려움을 겪는다면 다른 agent에게 해당 task를 위임하게된다.
    llm=llm
)

In [13]:
# task
research = Task(
    description="AI업게의 최신 기술 동향을 조사하고 요약을 제공하세요.",
    expected_output=("AI업계에서 가장 주목받는 3대 기술 개발동향과 그 중요성에 대한 신선한 관점을 요약한 글."),
    agent=researcher
)

write = Task(
    description="""테크 트랜드 연구원의 요약을 바탕으로 AI산업에 대한 매력적인 테크 뉴스레터를 작성하세요.
                테크 뉴스레터이므로 전문적인 용어를 사용해도 괜찮습니다. 답변에 있어서는 출처를 남겨주세요.
    """,
    expected_output="최신 기술 관련 소식을 재밌는 말투로 소개하는 4문단짜리 마크다운 형식 뉴스레터",
    agent=writer,
    output_file="./new_post.md"
)

In [14]:
# crew
crew = Crew(
    agents=[researcher, writer],
    tasks=[research, write],
    verbose=True,
    process=Process.sequential
)
# execute task
result = crew.kickoff()

2024-09-19 23:48:37,332 - 22888 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: 테크 트렌드 연구원
## Task: AI업게의 최신 기술 동향을 조사하고 요약을 제공하세요.


# Agent: 테크 트렌드 연구원
## Thought: 고려해야 할 최신 AI 기술 동향이 무엇인지 파악하기 위해 현재 AI 업계의 트렌드를 조사해야 합니다.
## Using tool: Search the internet
## Tool Input: 
{
  "search_query": "2024 AI 기술 동향"
}
## Tool Output: 

Search results: Title: 17개 산업별 2024년 AI 전망 - NVIDIA Blog Korea
Link: https://blogs.nvidia.co.kr/blog/2024-ai-predictions/
Snippet: 2024년부터는 생성형 AI를 사용해 비식별 데이터를 활용, LLM을 구축하고 맞춤화하기 시작할 것입니다. 기업은 AI 기반 슈퍼컴퓨팅을 통해 채팅, 동영상, 코드 등 비정형 ...
---
Title: 2024년에 주목해야 하는 AI 트렌드
Link: https://www.irsglobal.com/bbs/rwdboard/21651
Snippet: 2024년에는 AI 챗봇이 문의에 대한 즉각적인 답변, 복잡한 문제 해결, 더욱 폭넓은 리퀘스트에 대한 대응 등 보다 정련된 맞춤 서포트를 제공하게 된다. ...
---
Title: 2024년 주목해야 할 5가지 AI 트렌드 | Blog Home
Link: https://blog.purestorage.com/ko/perspectives-ko/unveiling-the-top-5-ai-trends-of-2024/
Snippet: 트렌드 5: AI 규제 – 법적 경계 탐색 · 트렌드 4: 윤리적 AI – 인공지능을 위한 도덕적 나침반 · 트렌드 3: 엣지에서의 AI – 최전방에 인텔리전스 도입.
---
Title: 2024 업무 동향 지수 – AI 현황 - 클루커스
Link: https://www.cloocus.com/insight-20

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  2.02it/s]




# Agent: 테크 트렌드 연구원
## Using tool: Search in a specific website
## Tool Input: 
{
  "search_query": "2024 AI 트렌드",
  "website": "https://blog.purestorage.com/ko/perspectives-ko/unveiling-the-top-5-ai-trends-of-2024/"
}
## Tool Output: 
Relevant Content:
2024년 주목해야 할 5가지 AI 트렌드 | Blog Home Subscribe Blog Home 기고 2024년 주목해야 할 5가지 AI 트렌드 2024년은 AI 여정에서 결정적인 순간이 될 것입니다. 이 글에서는 올해의 주요 AI 트렌드 5가지를 카운트다운 형식으로 살펴봅니다. 6분 2월 5, 2024 0 Share: Share on LinkedIn Share on X Share on Facebook AI and Machine Learning 이 글은 Perplexity AI의 지원을 받아 멜로디 자카리아스(Melody Zacharias)가 ‘Better Together’ 블로그를 위해 작성했습니다. 인공지능(AI) 환경은 전례 없는 속도로 진화하며 산업, 사회, 그리고 우리의 일상 생활을 바꾸어 놓고 있습니다. 2024년은 AI 여정에서 결정적인 순간이 될 것이며, 앞으로 지속적으로 영향을 줄 혁신적인 발전이 있을 것입니다. 지금부터 살펴볼 2024년에 주목해야 할 5가지 AI 트렌드는 우리가 마주하고 있는 AI의 향방을 정하는 데 중요한 역할을 할 것입니다. 트렌드 5: AI 규제 – 법적 경계 탐색 AI가 다양한 부문에 통합되면서 성숙해짐에 따라, 이제 포괄적인 법적 틀이 필요한 시점이 되었습니다. 이는 AI 규제가 혁신과 책임 있는 사용 사이의 균형을 맞추는 데 중요한 문제가 될 것임을 의미합니다. 전 세계 정부들은 AI 기술의 구현을 통제하기 위한 명확한 지침이 필요하다는 사실을 인지하고 있습니다

In [9]:
from IPython.display import display, Markdown
display(Markdown(result.raw))

# 인공지능 뉴스레터: 2024년 9월호

안녕하세요, AI 열혈 팬 여러분! 🚀 오늘은 인공지능(AI) 업계의 최신 트렌드와 기술 발전을 살펴보며, 여러분의 지식과 관심을 더욱 풍성하게 만들어드릴 시간을 가져보려고 합니다. 자, 그럼 우리가 알아야 할 핫한 주제들에 대해 깊이 들어가 볼까요?

## 생성적 AI의 진화, 한계를 넘어! 

최근 몇 년 동안 생성적 AI가 큰 주목을 받고 있죠. OpenAI의 GPT-4, DALL-E, Google의 Bard 같은 최첨단 모델들이 우리의 상상력을 뛰어넘는 콘텐츠를 생성하는데 일조하고 있습니다. 이들 기술은 단순히 재미있는 이미지를 만들어내는 것을 넘어서, 개인화된 마케팅과 고객 서비스의 혁신을 이끌고 있어요. 덕분에 기업들은 생산성을 높이고, 콘텐츠 제작에 드는 비용을 획기적으로 절감할 수 있는 기회를 찾고 있습니다. 마치 마케팅의 마법사가 된 듯한 기분이 드는 건 저만의 착각일까요? 🎨✨

## 클라우드와 AI의 시너지, 무한한 가능성!

AI 기술이 발전함에 따라 클라우드 컴퓨팅과의 통합도 가속화되고 있습니다. AWS, 구글 클라우드, 마이크로소프트 애저와 같은 클라우드 거물들이 AI 서비스를 강화하며, 기업들이 복잡한 AI 모델을 구축하는 비용과 시간을 현저히 줄이도록 돕고 있습니다. 이제는 지역적인 한계를 넘어, 글로벌한 데이터 분석과 AI 모델 배포가 가능해졌죠. 기업들은 클라우드를 통해 데이터의 바다 속에서 진주 같은 인사이트를 발견할 수 있게 되었습니다. 이건 마치 AI의 세계에서 가장 스마트한 여행가가 된 기분입니다! 🌍👍

## 책임 있는 AI 사용: 윤리와 규제가 필수

AI가 발전하면서 윤리 문제와 규제 필요성이 부각되고 있습니다. EU의 디지털 서비스법(DSA)과 미국의 AI 행동 강령 같은 법안들은 이제 더 이상 선택이 아닌 필수가 되고 있죠. 이러한 규제들은 사용자의 개인정보 보호와 AI의 결과물 신뢰성을 높이기 위해 필요합니다. 기업들은 이러한 책임 있는 사용을 통해 소비자 신뢰를 얻고, 지속 가능한 발전을 도모해야 합니다. 결국, 데이터와 사람을 존중할 줄 아는 AI가 되어야 하니까요! 🛡️🤖 

결론적으로, 2024년 현재 AI 업계는 생성적 AI의 발전, 클라우드와의 밀접한 통합, 그리고 철저한 윤리 및 규제 강화라는 세 가지 주제로 요약될 수 있습니다. 이러한 흐름들은 기업들에게 새로운 기회를 제공함과 동시에, 책임 있는 AI 생태계를 구축하는 데 기여하고 있습니다. AI가 그려갈 미래가 더욱 기대되는 오늘입니다! 다음 호에 뵐 때까지 안전하고 스마트한 하루 되세요! 🌟

*출처: 최신 AI 기술 동향 연구 (2024)*